### **Summations in Pyomo**

$$
\begin{alignat*}{3}
\text{minimize  }  & \sum_{i \in I} C(i)*P(i) && \\
\text{subject to  }
& \sum_{i \in I} P(i) = \sum_{s \in S} P_d(s)\\
& P_d(1) \le P(1) + P(4)\\
& 0 \le P(i) 
&& \qquad \forall i \in I \\
& P(i) \le P(i)^{lim}
&& \qquad \forall i \in I
\end{alignat*}
$$

In [1]:
import pyomo.environ as pe
import pyomo.opt as po

In [2]:
m = pe.ConcreteModel()

In [3]:
# Sets
m.I = pe.RangeSet(1, 5)
m.S = pe.RangeSet(1, 3)

_cost = {
    1: 0.1,
    2: 0.05,
    3: 0.3,
    4: 0.4,
    5: 0.01
}
m.C = pe.Param(m.I, initialize=_cost)

_powergen = {
    1: 20,
    2: 10,
    3: 40,
    4: 50,
    5: 5
}
m.P_lim = pe.Param(m.I, initialize=_powergen)

_demand = {
    1: 50,
    2: 20,
    3: 30 
}
m.Pd = pe.Param(m.S, initialize=_demand)

In [4]:
m.P = pe.Var(m.I, domain=pe.NonNegativeReals)

In [5]:
# Constraints
def _UP_lim(m, i):
    return m.P[i] <= m.P_lim[i]
m.UP_lim = pe.Constraint(m.I, rule=_UP_lim)

m.sum_cons = pe.Constraint(expr=(
    sum(m.Pd[s] for s in m.S) == sum(m.P[i] for i in m.I)
))

m.cons = pe.Constraint(expr=(
    m.Pd[1] <= m.P[1] + m.P[4]
))

expr = sum(m.C[i] * m.P[i] for i in m.I)
m.objective = pe.Objective(sense=pe.minimize, expr=expr)

In [6]:
solver = po.SolverFactory('glpk')
results = solver.solve(m)

In [7]:
m.P.display()

P : Size=5, Index=I
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :  20.0 :  None : False : False : NonNegativeReals
      2 :     0 :  10.0 :  None : False : False : NonNegativeReals
      3 :     0 :  35.0 :  None : False : False : NonNegativeReals
      4 :     0 :  30.0 :  None : False : False : NonNegativeReals
      5 :     0 :   5.0 :  None : False : False : NonNegativeReals
